In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Test with default hyperparameters

In [ ]:
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = CatBoostClassifier(silent=True, eval_metric='TotalF1')
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('09-catboost','CatBoostClassifier','1-Default', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('09-catboost','CatBoostClassifier','1-Default', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)



# Hyperparameters tuning

In [ ]:
%%script false
from scipy.stats import randint, uniform

modelClf = CatBoostClassifier(silent=True, eval_metric='TotalF1')

dic_param={
    'depth': randint(4, 12),
    'learning_rate': uniform(0.01, 0.3),
    'iterations': randint(100, 1250),
    'l2_leaf_reg': randint(1, 10),
    'border_count': randint(1, 300),
    'bagging_temperature': uniform(0.0, 1.0),
    'random_strength': uniform(0.0, 1.0)
}

res=fd.hyperparameterSelectionRandomizedSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)


In [ ]:
%%script false

modelClf = CatBoostClassifier(silent=True, eval_metric='TotalF1')

dic_param={
    'depth': [7,8,9,10,11],
    'learning_rate': [0.2],
    'iterations': [900,1000,1100,1200],
    'l2_leaf_reg': [1,2,3],
    'border_count': [245,255,265],
    'bagging_temperature': [0.9],
    'random_strength':[0.5]
}

#
#{'bagging_temperature': 0.5, 'border_count': 290, 'depth': 5, 'iterations': 1200, 'l2_leaf_reg': 1, 'learning_rate': 0.2, 'random_strength': 0.4}
#0.15591392405063292
#scoref1 0.5335276967930029

res=fd.hyperparameterSelectionGridSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)



In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
parameters= {'bagging_temperature': 0.2, 'border_count': 205, 'depth': 7, 'iterations': 1100, 'l2_leaf_reg': 1, 'learning_rate': 0.2, 'random_strength': 0.8}


modelClf = CatBoostClassifier(silent=True, eval_metric='TotalF1')
modelClf.set_params(**parameters)
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)

resultMd.update_time_response_result('09-catboost','CatBoostClassifier','2-After tuning', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('09-catboost','CatBoostClassifier','2-After tuning', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)

## result  (26/01/2025)
#test f1 score: 0.1286 mcc score: 0.1592 roc auc score: 0.5403

In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result('09-catboost','CatBoostClassifier','2-After tuning', f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])
